<a href="https://colab.research.google.com/github/wajidminhas/AI-Agents/blob/main/RAG-PINECONE/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [3]:
import getpass, time, os
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api  = userdata.get("PINECONE-API")

pc = Pinecone(api_key = pinecone_api)

In [6]:
import time

index_name = "test-index-feb-28"  # change if desired

# Check if the index already exists
if index_name in pc.list_indexes().names(): # Added () to call the .names property
    print(f"Index '{index_name}' already exists. Connecting to existing index.")
    index = pc.Index(index_name)
else:
    print(f"Index '{index_name}' does not exist. Creating new index.")
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    index = pc.Index(index_name)

Index 'test-index-feb-28' already exists. Connecting to existing index.


In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [8]:
vector = embeddings.embed_query("going to build rag")

In [9]:
vector[:4]

[0.03360629826784134,
 -0.012450501322746277,
 -0.032498568296432495,
 -0.024020353332161903]

In [10]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)